## Spark ##

* Procesamiento distruibuido de datos de grandes volúmenes utilizando un cluster.

__Cluster:__ Conjunto de computadoras que trabajan juntas y pueden ser vistas como un sistema único. Cada computadora del cluster en un nodo.

* Almacenamiento distribuido:

__FileSystem distribuido:__ Se encarga de gestionar cómo y donde guardar información en una computadora, y cómo poder consultarla. Almacena grandes volúmenes de datos es varios equipos, replica datos, tolerante a fallos, alta disponibilidad, relativo bajo costo.

__RDD:__  Resilient distributed dataset, which is a fault-tolerant collection of elements that can be operated on in parallel.

<https://spark.apache.org/docs/latest/rdd-programming-guide.html>

### Map-Reduce ###

* Se especifica una función map que procesa un par clave/valor para generar un conjunto intermedio de pares clave/valor.
* Se especifica una función reduce que combina todos los valores asociados a la misma clave. Deben ser conmutativas y asociativas.

__Shuffle & Sort:__ Mueve la salida de un preoceso map a un cierto equipo de tal forma que el un reducer pueda procesar registro.


In [2]:
import findspark
findspark.init()
import pyspark

from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
import pandas as pd

In [3]:
#create the Spark Session
spark = SparkSession.builder.getOrCreate()
#create the Spark Context
sc = spark.sparkContext

#### Paralelizar una colección ####

In [121]:
#1000 enteros en una lista
integers = range(1,1001)
len(integers)

1000

In [122]:
#Paralelizamos la coleccion utilizando 8 particiones o slices
#Esta operacion es una transformacion de datos en un RDD
#Dado que Spark usa lazy evaluation, no corren jobs de Spark hasta el momento
integers_rdd = sc.parallelize(integers, 8)
type(integers_rdd)

pyspark.rdd.PipelinedRDD

In [6]:
#Número de particiones
integers_rdd.getNumPartitions()

8

In [7]:
#Conjunto de transformaciones que se aplica
integers_rdd.toDebugString()

b'(8) PythonRDD[1] at RDD at PythonRDD.scala:48 []\n |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475 []'

In [119]:
#Métodos disponibles 
#help(integers_rdd)

In [9]:
#Primeros 5 registros
integers_rdd.take(5)

[1, 2, 3, 4, 5]

In [10]:
#Cantidad de elementos
integers_rdd.count()

1000

#### Acciones ####

* Count

In [11]:
integers_rdd.count()

1000

* Take

In [12]:
integers_rdd.take(5)

[1, 2, 3, 4, 5]

* Collect

Trae todos los registros de un rdd. No recomendado ya que puede sobrecargar el driver.

In [118]:
#integers_rdd.collect()

* First

In [14]:
integers_rdd.first()

1

* TakeOrdered

Primeros n registros en un orden indicado.

In [15]:
integers_rdd.takeOrdered(5, key=lambda x: -x)

[1000, 999, 998, 997, 996]

* TakeSample

Muestra de n registros aleatorios con o si reemplazo.

In [16]:
integers_rdd.takeSample(False, 5)

[115, 17, 667, 430, 363]

* Reduce

Obtiene un solo registro combinando el resultado en base a una función:

In [17]:
#Suma de todos los elementos
integers_rdd.reduce(lambda a, b: a + b)

500500

In [18]:
#Máximo
integers_rdd.reduce(lambda a, b: a if a > b else b)

1000

* CountByKey

Cuenta ocurrencias por cada clave.

En Spark para que un registro sea considerado con clave debe se una tupla de unicamente dos elementos. El primer elemento es la key y el segundo el valor. A su vez, la key y el valor pueden estar compuestos por tuplas.

In [19]:
integers_rdd.map(lambda x: (x % 2, 1)).countByKey()

defaultdict(int, {0: 500, 1: 500})

#### Transformaciones ####

* Map

Transforma cada registro en base a la función dada.

In [20]:
integers_rdd.map(lambda x: x * 2).take(5)

[2, 4, 6, 8, 10]

In [21]:
integers_rdd.map(lambda x: (x % 2, x)).take(5)

[(1, 1), (0, 2), (1, 3), (0, 4), (1, 5)]

* Filter

In [22]:
integers_rdd.filter(lambda x: x % 2 == 0).take(5)

[2, 4, 6, 8, 10]

In [23]:
integers_rdd.filter(lambda x: x % 2 == 0).count()

500

* FlatMap

Por cada registro puede generar 0 o mas registros.

In [24]:
integers_flat = integers_rdd.flatMap(lambda x: [(x), (x - 1), (x + 1)])
integers_flat.count()

3000

In [25]:
integers_flat.take(20)

[1, 0, 2, 2, 1, 3, 3, 2, 4, 4, 3, 5, 5, 4, 6, 6, 5, 7, 7, 6]

* ReduceByKey

Combina los registros para una misma clave en base a una función de reduce.

La función de reduce debe ser conmutativa y asociativa.

In [26]:
integers_flat.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).count()

1002

In [30]:
integers_flat.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).take(5)

[(0, 1), (256, 3), (912, 3), (520, 3), (648, 3)]

In [31]:
integers_flat.map(lambda x: (x, 1)).reduceByKey(lambda a, b: a + b).reduce(lambda a, b: a if a > b else b)

(1001, 1)

* GroupByKey

Agrupa los registros para cada clave. Es similar a reduceByKey pero con groupByKey se obtiene todos los registros para cada clave.

Solo se debe utilizar si es necesario la información de cada registro y la cantidad de registros por clave no es demasiado grande.

GroupByKey es una transformación costosa.

Si se desea realizar una agregación, usar reduceByKey. Usar groupByKey para hacer una agregación esta MAL.


In [120]:
integers_rdd.map(lambda x: (x % 2, x)).groupByKey().map(lambda x: (x[0], list(x[1]))).take(20)

* Distinct

Elimina registros duplicados.

In [34]:
integers_flat.count()

3000

In [35]:
integers_flat.distinct().count()

1002

#### Transformaciones ####

* Union

In [36]:
integers_2 = range(501, 1501)
len(integers_2)

1000

In [37]:
integers_rdd_2 = sc.parallelize(integers_2)

In [38]:
integers_rdd_2.count()

1000

In [39]:
integers_rdd.count()

1000

In [40]:
union = integers_rdd.union(integers_rdd_2)

In [41]:
union.take(5)

[1, 2, 3, 4, 5]

In [42]:
union.count()

2000

* Intersection

In [43]:
intersection = integers_rdd.intersection(integers_rdd_2)

In [45]:
intersection.count()

500

In [47]:
intersection.takeOrdered(10)

[501, 502, 503, 504, 505, 506, 507, 508, 509, 510]

* Subtract

In [48]:
subtract = integers_rdd.subtract(integers_rdd_2)

In [49]:
subtract.count()

500

In [50]:
subtract.collect()

[256,
 384,
 32,
 112,
 368,
 480,
 352,
 80,
 208,
 96,
 336,
 464,
 128,
 448,
 64,
 160,
 496,
 48,
 304,
 176,
 432,
 192,
 288,
 320,
 416,
 224,
 16,
 144,
 272,
 240,
 400,
 1,
 321,
 257,
 273,
 17,
 97,
 177,
 385,
 65,
 241,
 369,
 337,
 497,
 401,
 225,
 353,
 481,
 129,
 81,
 209,
 161,
 145,
 113,
 433,
 449,
 193,
 49,
 305,
 465,
 33,
 289,
 417,
 258,
 162,
 2,
 274,
 34,
 50,
 402,
 386,
 338,
 114,
 242,
 18,
 498,
 130,
 98,
 226,
 146,
 418,
 82,
 370,
 66,
 194,
 354,
 322,
 290,
 450,
 210,
 482,
 178,
 306,
 466,
 434,
 179,
 307,
 435,
 35,
 291,
 419,
 259,
 99,
 51,
 19,
 147,
 275,
 403,
 3,
 323,
 355,
 131,
 387,
 115,
 499,
 163,
 483,
 371,
 83,
 211,
 339,
 467,
 67,
 227,
 195,
 451,
 243,
 68,
 196,
 388,
 52,
 180,
 292,
 308,
 324,
 420,
 100,
 20,
 276,
 404,
 212,
 4,
 148,
 132,
 260,
 164,
 244,
 372,
 436,
 116,
 452,
 356,
 484,
 36,
 228,
 84,
 340,
 468,
 500,
 213,
 261,
 341,
 277,
 5,
 37,
 453,
 53,
 437,
 181,
 69,
 309,
 229,
 165,
 101

#### Join ####

Combina dos RDD en base a las claves de los registros. Junta cada registro del primer RDD con cada registro del segundo RDD con la misma clave. 

In [51]:
data_alumnos = [
  (1,'Damian'),
  (2,'Luis'),
  (3,'Martin'),
  (4,'Natalia'),
  (5,'Joaquin')
]

alumnos = sc.parallelize(data_alumnos)

In [52]:
alumnos.collect()

[(1, 'Damian'), (2, 'Luis'), (3, 'Martin'), (4, 'Natalia'), (5, 'Joaquin')]

In [53]:
data_materias_aprobadas = [
  (1, 'Algebra'),
  (2, 'Análisis Matemático'),
  (200, 'Algebra'),
  (2, 'Física')
]

materias_aprobadas = sc.parallelize(data_materias_aprobadas)

In [54]:
materias_aprobadas.collect()

[(1, 'Algebra'), (2, 'Análisis Matemático'), (200, 'Algebra'), (2, 'Física')]

* Inner Join

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) con todos los pares de elementos para cada clave que está en ambos sets simultaneamente.

In [55]:
alumnos.join(materias_aprobadas).collect()

[(1, ('Damian', 'Algebra')),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física'))]

* Left Outer Join

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurándonos que todos los datos del set de datos izquierdo estaran en el resultado del join.

In [56]:
alumnos.leftOuterJoin(materias_aprobadas).collect()

[(1, ('Damian', 'Algebra')),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (3, ('Martin', None)),
 (4, ('Natalia', None)),
 (5, ('Joaquin', None))]

* Right Outer Join

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurándonos que todos los datos del set de datos derecho estaran en el resultado del join.

In [57]:
alumnos.rightOuterJoin(materias_aprobadas).collect()

[(1, ('Damian', 'Algebra')),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (200, (None, 'Algebra'))]

* Outer/Full Join

Cuando se llama para sets de datos del tipo (K,V) y (K,W) devuelve un set de datos del tipo (K, (V,W)) asegurándonos que todos los datos de ambos set de datos estaran aunque no haya match de keys.

In [58]:
alumnos.fullOuterJoin(materias_aprobadas).collect()

[(1, ('Damian', 'Algebra')),
 (2, ('Luis', 'Análisis Matemático')),
 (2, ('Luis', 'Física')),
 (3, ('Martin', None)),
 (4, ('Natalia', None)),
 (5, ('Joaquin', None)),
 (200, (None, 'Algebra'))]

* Broadcast Join

__Variable Broadcast:__ Nos permite mantener una variable solo lectura cacheada en cada una de las maquinas del cluster en vez de enviar esa informacion con cada una de las tareas que se envian al cluster.
Esto es particularmente util cuando cuando tareas a partir de multiples etapas (stages) necesitan la misma información o cuando cachear información de forma deserializada es importante.

Tener en cuenta que esto es posible cuando uno de los data sets o conjunto de datos es lo suficientemente pequeño para ser broadcasteado a todos los nodos/workers del cluster.

Cuando un valor es "broadcasteado" al cluster, este es copiado a los nodos/workers sólo una vez (en vez de múltiples veces si la información fuera a enviarse en cada task). De esta forma se resuelve la consulta más rapido.

In [59]:
#RDD de productos por sus IDs identificando ventas de los mismos
prods_data = [1, 11, 1, 4, 5, 11, 2, 3, 4, 5, 6, 4, 5, 4, 3, 2, 1, 11, 2, 3, 4, 5, 6, 4, 3, 2, 1, 1]
prods = sc.parallelize(prods_data,3)

In [76]:
# Hash con los productos y sus nombres
product_names = {1:'papas',
                2:'cebollas',
                3:'tomates',
                4:'zanahorias',
                5:'batatas',
                6:'peras',
                7:'cilantro',
                8:'apio',
                9:'morrones',
                10:'manzanas',
                11:'naranjas'}

#Broadcast de la variable
bproduct_names = sc.broadcast(product_names)

In [77]:
#Productos que se vendieron más de 4 veces
popular_prods = prods.map(lambda x: (x, 1))\
                .reduceByKey(lambda x, y: x + y)\
                .filter(lambda x: x[1] >= 4)
        
popular_prods.collect()

[(3, 4), (1, 5), (4, 6), (2, 4), (5, 4)]

El join se realiza de forma implícita usando un map y dentro del mismo accediendo a la información de la variable a la que se realizo el broadcast via .value

In [78]:
popular_prods = popular_prods.map(lambda x: (bproduct_names.value[x[0]], x[0], x[1]))

In [79]:
popular_prods.take(5)

[('tomates', 3, 4),
 ('papas', 1, 5),
 ('zanahorias', 4, 6),
 ('cebollas', 2, 4),
 ('batatas', 5, 4)]

#### Transformaciones entre dos RDD ####

In [84]:
numbers = sc.parallelize(range(1, 11))
numbers.getNumPartitions()

8

In [85]:
sc.defaultParallelism

8

In [86]:
numbers.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

* Glom

Junta los registros de cada partición en una lista.

In [87]:
numbers.glom().collect()

[[1], [2], [3], [4, 5], [6], [7], [8], [9, 10]]

* MapPartitions

Devuelve un nuevo RDD aplicando una función a cada partición del RDD.

In [88]:
def f(iterator): yield __builtin__.sum(iterator)
numbers.mapPartitions(f).collect()

[1, 2, 3, 9, 6, 7, 8, 19]

* Repartition

Reshuffle los datos en el RDD de forma aleatoria para crear más o menos particiones y balancearlas.

Hace un shuffle de todo los datos por la red.

In [90]:
numbers = sc.parallelize(range(1, 11), 4)
numbers.getNumPartitions()

4

In [91]:
numbers.glom().collect()

[[1, 2], [3, 4, 5], [6, 7], [8, 9, 10]]

In [92]:
numbers_2 = numbers.repartition(2)

In [93]:
numbers_2.getNumPartitions()

2

In [94]:
numbers_2.glom().collect()        

[[1, 3, 6, 8], [2, 4, 5, 7, 9, 10]]

* Coalesce

Decrementa la cantidad de particiones del RDD.

No hace shuffle por defecto, solo pasa datos de una partición a otra.

No quedan balanceadas.

In [95]:
rdd_coalesce = numbers.coalesce(2)

In [96]:
rdd_coalesce.glom().collect()

[[1, 2, 3, 4, 5], [6, 7, 8, 9, 10]]

* RepartitionAndSortWithinPartitions

Reparticiona un RDD de acuerdo a un particionador y ordena los registros en base a su clave.

Los registros deben tener clave.

Es más eficiente que hacer un repartition y luego un sort dentro de cada partición ya que realiza el sort en el mismo paso de shuffle.

In [97]:
numbers.map(lambda x: (x, x)).collect()

[(1, 1),
 (2, 2),
 (3, 3),
 (4, 4),
 (5, 5),
 (6, 6),
 (7, 7),
 (8, 8),
 (9, 9),
 (10, 10)]

In [98]:
rdd.map(lambda x: (x, x)).glom().collect()

[[(1, 1)],
 [(2, 2)],
 [(3, 3)],
 [(4, 4), (5, 5)],
 [(6, 6)],
 [(7, 7)],
 [(8, 8)],
 [(9, 9), (10, 10)]]

* Ascending

In [99]:
#Son dos particiones, hace clave % 2 para elegir a donde va
numbers.map(lambda x: (x, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(2, 2), (4, 4), (6, 6), (8, 8), (10, 10)],
 [(1, 1), (3, 3), (5, 5), (7, 7), (9, 9)]]

In [100]:
numbers.map(lambda x: (x % 3, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(0, 3), (0, 6), (0, 9), (2, 2), (2, 5), (2, 8)],
 [(1, 1), (1, 4), (1, 7), (1, 10)]]

In [101]:
numbers.map(lambda x: (x % 3, x)).repartitionAndSortWithinPartitions(2, ascending=False).glom().collect()

[[(2, 2), (2, 5), (2, 8), (0, 3), (0, 6), (0, 9)],
 [(1, 1), (1, 4), (1, 7), (1, 10)]]

* PartitionFunc

In [102]:
#Todas las claves son múltiplo de 2, clave % 2 siempre es 0
#Ordena todas en la misma partición
numbers.map(lambda x: (x * 2, x)).repartitionAndSortWithinPartitions(2).glom().collect()

[[(2, 1),
  (4, 2),
  (6, 3),
  (8, 4),
  (10, 5),
  (12, 6),
  (14, 7),
  (16, 8),
  (18, 9),
  (20, 10)],
 []]

In [103]:
#Se puede cambiar la función de partición
numbers.map(lambda x: (x * 2, x)).repartitionAndSortWithinPartitions(2, partitionFunc=lambda x: (x % 3)).glom().collect()

[[(2, 1), (6, 3), (8, 4), (12, 6), (14, 7), (18, 9), (20, 10)],
 [(4, 2), (10, 5), (16, 8)]]

#### Persistencia de RDD ####

* Cache 

Cachea un RDD intermedio que va a ser utilizado varias veces de modo de evitar tener que ejecutar todas las transformaciones cada vez.

In [105]:
large = sc.parallelize(range(1, 100000))

In [106]:
large_cached = large.map(lambda x: x * 10).cache()

In [107]:
large_cached.count()

99999

In [108]:
large_cached.take(10)

[10, 20, 30, 40, 50, 60, 70, 80, 90, 100]

* SaveAsTextFile

Guarda un RDD a disco en un archivo de texto.

In [109]:
large.saveAsTextFile('numbers.txt')

In [113]:
large_text = sc.textFile('numbers.txt')

In [117]:
large_text.take(10)

['62500',
 '62501',
 '62502',
 '62503',
 '62504',
 '62505',
 '62506',
 '62507',
 '62508',
 '62509']

* SaveAsPickleFile

Guarda un RDD a disco en un archivo con los datos serializados.

In [112]:
large.saveAsPickleFile('numbers2.file')

In [115]:
large_serial = sc.pickleFile('numbers2.file/')

In [116]:
large_serial.take(10)

[62500, 62501, 62502, 62503, 62504, 62505, 62506, 62507, 62508, 62509]